In [2]:
import gc
import os
import sys
import time
import math

import cv2
import numpy as np
import torch.nn as nn

import pandas as pd
import torch
from PIL import Image
from torch import optim
from torch.optim.lr_scheduler import ExponentialLR, StepLR
from torchvision import transforms
import re
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import warnings
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch.nn.functional as F
from collections import defaultdict
from itertools import product

from spellchecker import SpellChecker

warnings.filterwarnings("ignore", category=UserWarning)  # Disable UserWarnings
tqdm.pandas()

In [3]:
train_df = pd.read_csv("./dataset/train.csv")
test_df = pd.read_csv("./dataset/test.csv")
validation_df = pd.read_csv("./dataset/val.csv")

In [4]:
spell = SpellChecker()

def text_preparation_with_spell_correction(sentence):
    # 1. Lowercase everything
    sentence = sentence.lower()

    # 2. Remove all symbols other than a-z.
    pattern = re.compile(r"[^a-z ]")
    sentence = re.sub(pattern, " ", sentence)

    # 3. Correct spelling
    words = sentence.split()
    corrected_words = []
    for word in words:
        corrected_word = spell.correction(word)
        if corrected_word is None:
            corrected_word = '<unk>'  # Use '<unk>' if no correction found
        corrected_words.append(corrected_word)
    corrected_sentence = ' '.join(corrected_words)
    
    return corrected_sentence

sentence_example = '"A domesticated carnivvorous mzammal that typicbally hfaas a lons sfnout, an acxujte sense off osmell, noneetractaaln crlaws, anid xbarkring,y howlingu, or whining rvoiche."'
print(text_preparation_with_spell_correction(sentence_example))

a domesticated carnivorous mammal that typically haas a long snout an acute sense off smell <unk> claws and barking y howling or whining voice


In [5]:
from Levenshtein import distance as levenshtein_distance

# Assume 'vocabulary' is a predefined list of words expected in your domain
vocabulary = ['example', 'list', 'of', 'words', 'in', 'your', 'vocabulary']


def find_closest_word(word, vocabulary):
    return min(vocabulary, key=lambda v: levenshtein_distance(word, v))


def text_preparation_with_levenshtein(sentence):
    # 1. Lowercase everything
    sentence = sentence.lower()

    # 2. Remove all symbols other than a-z.
    pattern = re.compile(r"[^a-z ]")
    sentence = re.sub(pattern, " ", sentence)

    # 3. Replace words with the closest in vocabulary
    words = sentence.split()
    closest_words = [find_closest_word(word, vocabulary) for word in words]
    corrected_sentence = ' '.join(closest_words)
    
    return corrected_sentence

In [6]:
# Compile the regular expression pattern
pattern = re.compile(r"[^a-z ]")


def text_preparetion_simple(sentence):
    # 1. Lowercase everything
    sentence = sentence.lower()

    # 2. Remove all symbols other than a-z.
    sentence = re.sub(pattern, " ", sentence)

    return sentence

sentence1 = "A World War II-era bomber flying out of formation"
text_preparetion_simple(sentence1)

'a world war ii era bomber flying out of formation'

In [7]:
train_df['preprocessed_text'] = train_df['caption'].progress_apply(text_preparation_with_spell_correction)
validation_df['preprocessed_text'] = validation_df['caption'].progress_apply(text_preparation_with_spell_correction)
test_df['preprocessed_text'] = test_df['caption'].progress_apply(text_preparation_with_spell_correction)

100%|██████████| 2000/2000 [05:11<00:00,  6.41it/s]


In [8]:
test_df

,id,caption,image_id,preprocessed_text
0,7f8da4d4-5d37-4b83-afe8-e424bf567813,A red car and a white sheep.,23648,a red car and a white sheep
1,0f50dd85-9e08-4e4b-bab0-35c2ff27a865,A single clock is sitting on a table.,103430,a single clock is sitting on a table
2,c4cf245e-33e0-42b2-a04c-7c657f2c537c,"A real life photography of super mario, 8k Ult...",471992,a real life photography of super mario k ultra he
3,5b149fa4-a764-49d3-8c5a-9ddf71d37ca7,A device consisting of a circular canopy of cl...,266533,a device consisting of a circular canopy of cl...
4,d15c5ddb-dd79-4ce8-bd95-6bae2d005c7c,Paying for a quarter-sized pizza with a pizza-...,534923,paying for a quarter sized pizza with a pizza ...
...,...,...,...,...
1995,1f4535d1-c6e2-4490-a364-a3165944e393,A small blue book sitting on a large red book.,476173,a small blue book sitting on a large red book
1996,31bf3172-8374-4b18-993f-a8529eb9f900,A yellow colored giraffe.,159627,a yellow colored giraffe
1997,362645ef-7f02-4f6d-891d-5a5207f0125b,An elephant is behind a tree. You can see the ...,131115,an elephant is behind a tree you can see the t...
1998,0b57e533-d8a6-4d9b-82f2-88e5faef1a8f,A domesticated carnivorous mammal that typical...,331137,a domesticated carnivorous mammal that typical...


In [9]:
def build_simple_vocab(sentences, special_tokens=None):
    """
    Manually create a vocabulary from a list of tokenized sentences.
    
    Args:
        sentences (list of str): List of sentences to build vocabulary from.
        special_tokens (list of str): Special tokens like <pad>, <unk>.
        
    Returns:
        dict: A vocabulary mapping tokens to indices.
        dict: An inverse vocabulary mapping indices to tokens.
    """
    special_tokens = special_tokens or ['<pad>', '<unk>']
    vocab = defaultdict(lambda: len(vocab))  # Default index is the current vocab size
    for token in special_tokens:
        vocab[token]  # Add special tokens first
    
    # Add tokens from sentences
    for sentence in sentences:
        for token in sentence.split():
            if token.strip():  # Exclude empty tokens
                vocab[token]
    
    # Convert to a normal dict (no longer dynamic)
    vocab = dict(vocab)
    inverse_vocab = {index: token for token, index in vocab.items()}
    return vocab, inverse_vocab


# Vectorize a sentence
def vectorize_sentence(sentence, vocab):
    """
    Converts a sentence into a tensor of token indices using a given vocabulary,
    ignoring empty tokens.
    
    Args:
        sentence (str): Input sentence.
        vocab (Vocab): Vocabulary to map tokens to indices.
        
    Returns:
        torch.Tensor: Vectorized sentence as a tensor.
    """
    # Ensure '<unk>' exists in the vocabulary
    unk_idx = vocab.get('<unk>', -1)
    if unk_idx == -1:
        raise ValueError("The vocabulary must include '<unk>' for unknown tokens.")
    

    # Split sentence into tokens and map them to indices
    tokens = [token for token in sentence.split() if token.strip()]
    return torch.tensor([vocab.get(token, unk_idx) for token in tokens], dtype=torch.long)


class GaussianBlurTransform:
    def __init__(self, kernel_size):
        self.kernel_size = kernel_size

    def __call__(self, img):
        img = np.array(img)
        img = cv2.GaussianBlur(img, (self.kernel_size, self.kernel_size), 0)
        return Image.fromarray(img)


# Custom Dataset Class
class PreprocessingDataset(Dataset):
    def __init__(self, dataframe, vocab, images_path, train=True, max_len=None):
        """
        Dataset for preprocessing image-text pairs.

        Args:
            dataframe (pd.DataFrame): DataFrame containing 'image_id', 'sentence', and optionally 'label'.
            vocab (Vocab): Vocabulary for text vectorization.
            train (bool): Whether this is a training dataset.
            max_len (int): Maximum length for sentences. If None, no truncation.
        """
        super().__init__()
        self.dataframe = dataframe
        self.vocab = vocab
        self.train = train
        self.max_len = max_len
        self.images_path = images_path

        # Define image transformations
        self.image_transform = transforms.Compose([
            # transforms.RandomResizedCrop(100, scale=(0.8, 1.0)),
            # transforms.RandomHorizontalFlip(),
            GaussianBlurTransform(kernel_size=3),
            # transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
            transforms.ToTensor(),
            # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]

        # Process sentence
        sentence = row['caption']
        vectorized_sentence = vectorize_sentence(sentence, self.vocab)

        # Pad or truncate the sentence
        if len(vectorized_sentence) < self.max_len:
            padding_length = self.max_len - len(vectorized_sentence)
            pad_tensor = torch.full((padding_length,), self.vocab['<pad>'], dtype=torch.long)
            vectorized_sentence = torch.cat((vectorized_sentence, pad_tensor), dim=0)
        else:
            vectorized_sentence = vectorized_sentence[:self.max_len]

        # Process image
        image_path = f"{self.images_path}{row['image_id']}.jpg"
            
        try:
            image = Image.open(image_path).convert("RGB")  # Convert to RGB
            image = self.image_transform(image)
        except FileNotFoundError:
            raise FileNotFoundError(f"Image not found at path: {image_path}")

        # Handle labels (for training)
        if self.train:
            label = row['label']
            label = torch.tensor(label, dtype=torch.long)
            return {
                'images': image,
                'captions': vectorized_sentence,
                'labels': label
            }
        else:
            return {
                'images': image,
                'captions': vectorized_sentence,
                'id': row['id']
            }

In [10]:
tokenized_sentence = [sentence.split(" ") for sentence in train_df['preprocessed_text']]
max_len = max(len([token for token in sentence.split(" ")]) for sentence in train_df['preprocessed_text'])
print(max_len)
vocab, inverse_vocab = build_simple_vocab(train_df['preprocessed_text'])

45


In [11]:
train_dataset = PreprocessingDataset(train_df, vocab, train=True, max_len=max_len, images_path = "./dataset/train_images/")
test_dataset = PreprocessingDataset(test_df, vocab, train=False, max_len=max_len, images_path = "./dataset/test_images/")
val_dataset = PreprocessingDataset(validation_df, vocab, train=True, max_len=max_len, images_path = "./dataset/val_images/")

In [12]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True )
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
class CNN(nn.Module):
    def __init__(self, cnn_dropout_value):
        super(CNN, self).__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(cnn_dropout_value),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Dropout(cnn_dropout_value),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Dropout(cnn_dropout_value),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        self.flatten = nn.Flatten()

    def forward(self, images):
        img_features = self.layers(images)  # (Batch, 128, 1, 1)
        img_features = self.flatten(img_features)  # (Batch, 128)
        return img_features


In [15]:
class PositionalEncoding(nn.Module):
    def __init__(self, embedding_dim, max_len):
        super(PositionalEncoding, self).__init__()
        # Create a matrix of shape (max_len, embedding_dim) for positional encodings
        position = torch.arange(0, max_len).unsqueeze(1)  # Shape: (max_len, 1)
        div_term = torch.exp(torch.arange(0, embedding_dim, 2) * (-math.log(float(max_len)) / embedding_dim))
        
        pe = torch.zeros(max_len, embedding_dim)  # Shape: (max_len, embedding_dim)
        pe[:, 0::2] = torch.sin(position * div_term)  # Sin for even indices
        pe[:, 1::2] = torch.cos(position * div_term)  # Cos for odd indices
        
        pe = pe.unsqueeze(0)  # Add batch dimension: (1, max_len, embedding_dim)
        self.register_buffer('pe', pe)  # Register as non-learnable buffer

    def forward(self, x):
        """
        Add positional encoding to the input embeddings.
        x: (Batch, SeqLen, EmbeddingDim)
        """
        seq_len = x.size(1)
        return x + self.pe[:, :seq_len, :].to(x.device)

In [16]:
class TextModule(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_heads, transformer_hidden_dim, num_transformer_layers, seq_len, transformer_dropout_value):
        super(TextModule, self).__init__()
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim, padding_idx=vocab["<pad>"])
        self.positional_encoding = PositionalEncoding(embedding_dim=embedding_dim, max_len=seq_len)
        
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=embedding_dim,
                nhead=num_heads,
                dim_feedforward=transformer_hidden_dim,
                dropout=transformer_dropout_value,
                activation='relu'
            ),
            num_layers=num_transformer_layers
        )

    def forward(self, captions):
        embedded_captions = self.embedding(captions)  # (Batch, SeqLen, EmbeddingDim)
        pos_encoded_captions = self.positional_encoding(embedded_captions)  # Apply positional encoding
        
        # Transformer expects input shape (SeqLen, Batch, EmbeddingDim)
        transformer_input = pos_encoded_captions.permute(1, 0, 2)  # (SeqLen, Batch, EmbeddingDim)
        transformer_output = self.transformer_encoder(transformer_input)  # (SeqLen, Batch, EmbeddingDim)
        text_features = transformer_output.mean(dim=0)  # Mean pooling over SeqLen: (Batch, EmbeddingDim)
        return text_features

In [17]:
class Head(nn.Module):
    def __init__(self, image_feature_dim, text_feature_dim, transformer_dropout_value, num_classes):
        super(Head, self).__init__()
        self.fc1 = nn.Linear(image_feature_dim + text_feature_dim, 256)  # Combine image + text features
        self.dropout = nn.Dropout(transformer_dropout_value)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, img_features, text_features):
        combined_features = torch.cat((img_features, text_features), dim=1)  # (Batch, 128 + EmbeddingDim)
        x = F.relu(self.fc1(combined_features))  # (Batch, 256)
        x = self.dropout(x)  # Apply dropout
        x = self.fc2(x)  # (Batch, num_classes)
        return x.squeeze(1)  # Logits (not probabilities)

In [18]:
class ImageTextClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_heads, transformer_hidden_dim, num_transformer_layers, seq_len, cnn_dropout_value, transformer_dropout_value, num_classes=1):
        super(ImageTextClassifier, self).__init__()
        self.cnn = CNN(cnn_dropout_value)
        self.text_module = TextModule(
            vocab_size, embedding_dim, num_heads,
            transformer_hidden_dim, num_transformer_layers,
            seq_len, transformer_dropout_value
        )
        self.head = Head(image_feature_dim=128, text_feature_dim=embedding_dim, transformer_dropout_value=transformer_dropout_value, num_classes=num_classes)

    def forward(self, images, captions):
        img_features = self.cnn(images)  # Image feature extraction
        text_features = self.text_module(captions)  # Text feature extraction
        return self.head(img_features, text_features)  # Classification

In [19]:
def initialize_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)
    elif isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
        if m.bias is not None:
            nn.init.zeros_(m.bias)
    elif isinstance(m, nn.Embedding):
        nn.init.xavier_uniform_(m.weight)
    elif isinstance(m, nn.LayerNorm):
        nn.init.ones_(m.weight)
        nn.init.zeros_(m.bias)
    elif isinstance(m, nn.TransformerEncoderLayer):
        for name, param in m.named_parameters():
            if "weight" in name:
                if param.dim() > 1:
                    nn.init.xavier_uniform_(param)
                else:
                    nn.init.ones_(param)
            elif "bias" in name:
                nn.init.zeros_(param)

In [20]:
# # Test model code
model = ImageTextClassifier(
    vocab_size=10000,  # Example parameters
    embedding_dim=128,
    num_heads=8,
    transformer_hidden_dim=512,
    num_transformer_layers=6,
    seq_len=max_len,
    cnn_dropout_value=0.3,
    transformer_dropout_value=0.3,
    num_classes=1
)

# Apply weight initialization recursively
model.apply(initialize_weights)

# Dummy input data
images = torch.randn(16, 3, 100, 100)  # Batch of 16 RGB images of size 224x224
captions = torch.randint(0, len(vocab), (16, max_len))  # Batch of 16 captions with max_len tokens each

output = model(images, captions)
print(output.shape)  # Should be (16) 

torch.Size([16])


In [48]:
# Initialize the model
model_config = {
    "vocab_size": len(vocab),
    "embedding_dim": 256,
    "num_heads": 4,
    "transformer_hidden_dim": 256,
    "num_transformer_layers": 8,
    "seq_len": max_len,
    "cnn_dropout_value": 0.5,
    "transformer_dropout_value": 0.5,
}
model = ImageTextClassifier(**model_config)
model.to(device)
model.apply(initialize_weights)

ImageTextClassifier(
  (cnn): CNN(
    (layers): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
      (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): Dropout(p=0.5, inplace=False)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU()
      (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): Dropout(p=0.5, inplace=False)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU()
      (12): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (13): Dropout(p=0.5, inplace=False)
      (14): AdaptiveAvgPool2d(output_size=(1, 1))
    )
    (flatten): Flatten(st

In [49]:
cnn_params = model.cnn.parameters()  # Parameters of the CNN module
text_params = model.text_module.parameters()  # Parameters of the text module
head_params = model.head.parameters()

cnn_optimizer = optim.AdamW(cnn_params, lr=1e-3, weight_decay=1e-4)
text_optimizer = optim.AdamW(text_params, lr=1e-5, weight_decay=1e-5)
head_optimizer = optim.AdamW(head_params, lr=5e-4, weight_decay=1e-4)

criterion = nn.BCEWithLogitsLoss()  # Use logits directly


In [50]:
cnn_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
    cnn_optimizer, 
    T_0=5,  # Number of epochs before the first restart
    T_mult=2,  # Multiplicative factor for increasing restart period
    eta_min=1e-4  # Minimum learning rate
)

text_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
    text_optimizer,
    T_0=5,  # Number of epochs before the first restart
    T_mult=2,  # Multiplicative factor for increasing restart period
    eta_min=1e-5  # Minimum learning rate
)

head_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
    head_optimizer,
    T_0=5,  # Number of epochs before the first restart
    T_mult=2,  # Multiplicative factor for increasing restart period
    eta_min=1e-4  # Minimum learning rate
)

In [51]:
import copy

def training_method(model, criterion, cnn_optimizer, text_optimizer, head_optimizer, cnn_scheduler, text_scheduler, head_scheduler, num_epochs, train_loader, val_loader, patience=5, delta = 0.2, loss_procentage_improvement=10):
    train_losses = []  # List to store training losses
    val_losses = []  # List to store validation losses
    val_accuracies = []  # List to store validation accuracies
    val_precisions = []  # List to store validation precisions
    val_recalls = []  # List to store validation recalls
    val_f1s = []  # List to store validation F1-scores
    learning_rates = [] # List to store learning rate progression

    best_val_loss = float('inf')  # Initialize the best validation loss
    initial_loss = float('inf')
    best_model = None  # Store the best model
    epochs_without_improvement = 0  # Track epochs without improvement

    for epoch in range(num_epochs):
        start_time = time.time()
        ### TRAINING
        model.train()
        training_loss = 0.0
        
        for batch in train_loader:
            images = batch['images'].to(device)      # Images from batch
            captions = batch['captions'].to(device)  # Captions from batch
            labels = batch['labels'].to(device).float()  # Binary labels (0/1), converted to float
            
            cnn_optimizer.zero_grad()  # Reset gradients
            text_optimizer.zero_grad()
            head_optimizer.zero_grad()
            
            output = model(images, captions)  # Forward pass (logits)
            loss = criterion(output, labels)  # Compute loss
            loss.backward()  # Backpropagation
            
            cnn_optimizer.step()  # Update weights
            text_optimizer.step()
            head_optimizer.step()
            
            training_loss += loss.item()  # Accumulate loss
            
        train_loss = training_loss / len(train_loader)  # Average training loss
        train_losses.append(train_loss)

        ### VALIDATING
        model.eval()
        validation_loss = 0.0
        all_labels = []  # Ground truth labels for validation
        all_preds = []   # Predictions for validation
        
        with torch.no_grad():
            for batch in val_loader:
                images = batch['images'].to(device)
                captions = batch['captions'].to(device)
                labels = batch['labels'].to(device).float()
                
                output = model(images, captions)  # Forward pass (logits)
                loss = criterion(output, labels)  # Compute validation loss
                validation_loss += loss.item()
                
                # Convert logits to probabilities and apply threshold
                preds = (torch.sigmoid(output) > 0.5).float()
                
                # Store for statistics
                all_labels.extend(labels.cpu().numpy())
                all_preds.extend(preds.cpu().numpy())
                
        val_loss = validation_loss / len(val_loader)  # Average validation loss
        val_losses.append(val_loss)
        
        # Compute validation statistics
        val_accuracy = accuracy_score(all_labels, all_preds)
        val_precision = precision_score(all_labels, all_preds, zero_division=0)
        val_recall = recall_score(all_labels, all_preds, zero_division=0)
        val_f1 = f1_score(all_labels, all_preds, zero_division=0)
        
        val_accuracies.append(val_accuracy)
        val_precisions.append(val_precision)
        val_recalls.append(val_recall)
        val_f1s.append(val_f1)
        
        if epoch == 1:
            initial_loss = val_loss

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = copy.deepcopy(model)  # Save the best model
            epochs_without_improvement = 0  # Reset counter
            print(f"New best model with Loss: {val_loss:.4f} at epoch {epoch + 1}")
        elif val_loss < best_val_loss + delta:
            print(f"Validation loss did not improve significantly")            
        else:
            epochs_without_improvement += 1
            print(f"Validation loss did not improve for {epochs_without_improvement} epoch(s).")
            # Stop training if validation loss does not improve for 'patience' epochs
            if epochs_without_improvement >= patience:
                print(f"Early stopping at epoch {epoch + 1}. Best Loss: {best_val_loss:.4f}")
                break  # Exit training loop


        # Step the learning rate scheduler
        cnn_scheduler.step()
        text_scheduler.step()
        head_scheduler.step()
        
        end_time = time.time()

        print(f"\nEpoch {epoch + 1}/{num_epochs} - "
              f"Training Loss: {train_loss:.4f} - "
              f"Validation Loss: {val_loss:.4f} - "
              f"Accuracy: {val_accuracy:.4f} - "
              f"Precision: {val_precision:.4f} - "
              f"Recall: {val_recall:.4f} - "
              f"F1 Score: {val_f1:.4f} - "
              f"Time: {end_time - start_time:.2f}")

    print('Training finished!')
    
    # save the model only if the best loss is lower than the first initial loss ( to see that the model actually improved with 10% loss )
    if best_val_loss < (100 - loss_procentage_improvement) * initial_loss:
        # Init plot&model save path
        plt_save_path = "models/"
        model_config['eval_loss'] = best_val_loss
        for key, value in model_config.items():
            plt_save_path += key + "=" + str(value) + "+"
        plt_save_path = plt_save_path[:-1] + ".png"
        model_path = plt_save_path[:-4] + ".pt"
        
        torch.save(best_model.state_dict(), model_path)
        print(f"Best model with Loss: {best_val_loss:.4f} saved.")
        print(f"Model saved to {model_path}")

        # Plotting the losses and validation metrics over epochs
        plt.figure(figsize=(12, 8))
        plt.subplot(2, 1, 1)
        plt.plot(train_losses, label='Training Loss')
        plt.plot(val_losses, label='Validation Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.title('Training and Validation Loss')
        plt.legend()
        
        plt.subplot(2, 1, 2)
        plt.plot(val_accuracies, label='Accuracy')
        plt.plot(val_precisions, label='Precision')
        plt.plot(val_recalls, label='Recall')
        plt.plot(val_f1s, label='F1 Score')
        plt.xlabel('Epochs')
        plt.ylabel('Metric')
        plt.title('Validation Metrics')
        plt.legend()

    
        plt.tight_layout()
        plt.savefig(plt_save_path)
        plt.show()
    
    else:
        print(f"Model wasn't saved because it didn't improve: {loss_procentage_improvement}%")

In [52]:
training_method(model, criterion, cnn_optimizer, text_optimizer, head_optimizer, cnn_scheduler, text_scheduler, head_scheduler, num_epochs=100, train_loader=train_dataloader, val_loader=val_dataloader)

New best model with Loss: 0.7390 at epoch 1

Epoch 1/100 - Training Loss: 0.7064 - Validation Loss: 0.7390 - Accuracy: 0.5160 - Precision: 0.6034 - Recall: 0.0933 - F1 Score: 0.1617 - Time: 12.25
New best model with Loss: 0.7187 at epoch 2

Epoch 2/100 - Training Loss: 0.6919 - Validation Loss: 0.7187 - Accuracy: 0.5143 - Precision: 0.5576 - Recall: 0.1387 - F1 Score: 0.2221 - Time: 12.31
Validation loss did not improve significantly

Epoch 3/100 - Training Loss: 0.6890 - Validation Loss: 0.7653 - Accuracy: 0.5133 - Precision: 0.5524 - Recall: 0.1407 - F1 Score: 0.2242 - Time: 12.17
Validation loss did not improve significantly

Epoch 4/100 - Training Loss: 0.6864 - Validation Loss: 0.8038 - Accuracy: 0.5103 - Precision: 0.5486 - Recall: 0.1167 - F1 Score: 0.1924 - Time: 12.25
Validation loss did not improve significantly

Epoch 5/100 - Training Loss: 0.6845 - Validation Loss: 0.7723 - Accuracy: 0.5203 - Precision: 0.5444 - Recall: 0.2493 - F1 Score: 0.3420 - Time: 12.44
Validation los

KeyboardInterrupt: 

In [24]:
ids = []
predictions = []

def make_submission(model, test_loader):
    model.eval()

    with torch.no_grad():
        for batch in tqdm(test_loader):
            images = batch['images'].to(device)
            captions = batch['captions'].to(device)
            id = batch['id']
            
            output = model(images, captions)
            preds = (torch.sigmoid(output) > 0.5).int()
            
            ids.extend(id)
            predictions.extend(preds.cpu().tolist())
    df = pd.DataFrame({'id': ids, 'label': predictions})
    df.to_csv('submission2.csv', index=False)

In [25]:
# model_config = {
#     "vocab_size": len(vocab),
#     "embedding_dim": 128,
#     "num_heads": 2,
#     "transformer_hidden_dim": 256,
#     "num_transformer_layers": 2,
#     "seq_len": max_len,
#     "cnn_dropout_value": 0.4,
#     "transformer_dropout_value": 0.5,
# }
# model = ImageTextClassifier(**model_config)
# model_path = "models/vocab_size=3733+embedding_dim=128+num_filters=128+filter_sizes=[3, 4, 5, 6, 7, 8]+seq_len=53+cnn_text_drop_value=0.5+cnn_dropout_value=0.4+transformer_dropout_value=0.5+num_classes=1+eval_loss=0.6212541232717798.pt"
# model.load_state_dict(torch.load(model_path, weights_only=True))

RuntimeError: Error(s) in loading state_dict for ImageTextClassifier:
	Missing key(s) in state_dict: "text_module.transformer_encoder.layers.0.self_attn.in_proj_weight", "text_module.transformer_encoder.layers.0.self_attn.in_proj_bias", "text_module.transformer_encoder.layers.0.self_attn.out_proj.weight", "text_module.transformer_encoder.layers.0.self_attn.out_proj.bias", "text_module.transformer_encoder.layers.0.linear1.weight", "text_module.transformer_encoder.layers.0.linear1.bias", "text_module.transformer_encoder.layers.0.linear2.weight", "text_module.transformer_encoder.layers.0.linear2.bias", "text_module.transformer_encoder.layers.0.norm1.weight", "text_module.transformer_encoder.layers.0.norm1.bias", "text_module.transformer_encoder.layers.0.norm2.weight", "text_module.transformer_encoder.layers.0.norm2.bias", "text_module.transformer_encoder.layers.1.self_attn.in_proj_weight", "text_module.transformer_encoder.layers.1.self_attn.in_proj_bias", "text_module.transformer_encoder.

In [ ]:
# make_submission(model, test_dataloader)

In [26]:

def hyperparameter_tuning(vocab_size, max_len, train_loader, val_loader, param_grid, training_method, num_epochs=100):

    # Create all combinations of hyperparameters
    keys, values = zip(*param_grid.items())
    param_combinations = [dict(zip(keys, v)) for v in product(*values)]
    
    for params in tqdm(param_combinations):
        print(f"Testing configuration: {params}")
        
        try:
            # Update model configuration
            model_config = {
                "vocab_size": vocab_size,
                "embedding_dim": params["embedding_dim"],
                "num_heads": params["num_heads"],
                "transformer_hidden_dim": params["transformer_hidden_dim"],
                "num_transformer_layers": params["num_transformer_layers"],
                "seq_len": max_len,
                "cnn_dropout_value": params["cnn_dropout_value"],
                "transformer_dropout_value": params["transformer_dropout_value"],
            }
            
            # Initialize model
            model = ImageTextClassifier(**model_config)
            model.to(device)
            model.apply(initialize_weights)
            
            # Define criterion, optimizer, and scheduler
            criterion = nn.BCEWithLogitsLoss()
            optimizer = optim.AdamW(
                model.parameters(), 
                lr=params["lr"], 
                weight_decay=params["weight_decay"]
            )
            scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
                optimizer, 
                T_0=params["T_0"]
            )
            
            # Train the model
            training_method(
                model, criterion, optimizer, scheduler,
                num_epochs=num_epochs,
                train_loader=train_loader,
                val_loader=val_loader
            )
            print(f"Completed configuration: {params}")
        
        except RuntimeError as e:
            print(f"Error with configuration: {params}")
            print(f"Error message: {str(e)}")
        
        finally:
            # Reset GPU memory
            print("Resetting GPU memory...")
            torch.cuda.empty_cache()
            gc.collect()

In [27]:
param_grid = {
    "embedding_dim": [128],
    "num_heads": [4],
    "transformer_hidden_dim": [512],
    "num_transformer_layers": [8],
    "cnn_dropout_value": [0.5],
    "transformer_dropout_value": [0.7],
    "lr": [5e-4, 5e-5],
    "weight_decay": [1e-3, 1e-4],
    "T_0": [20],
    "T_mult": [1],
    "eta_min": [5e-6],
}

total_combinations = math.prod(len(values) for values in param_grid.values())
print(f"Total combinations: {total_combinations}")

time_per_epoch = 11  # seconds
num_epochs = 100  # epochs per configuration
total_time_seconds = total_combinations * time_per_epoch * num_epochs

# Convert to hours
total_time_hours = total_time_seconds / 3600
print(f"Total time to hyper tune: {total_time_hours} hours")

Total combinations: 4
Total time to hyper tune: 1.2222222222222223 hours


In [ ]:
results = hyperparameter_tuning(
    vocab_size=len(vocab),
    max_len=max_len,
    train_loader=train_dataloader,
    val_loader=val_dataloader,
    param_grid=param_grid,
    training_method=training_method,
    num_epochs=100
)

  0%|          | 0/4 [00:00<?, ?it/s]

Testing configuration: {'embedding_dim': 128, 'num_heads': 4, 'transformer_hidden_dim': 512, 'num_transformer_layers': 8, 'cnn_dropout_value': 0.5, 'transformer_dropout_value': 0.7, 'lr': 0.0005, 'weight_decay': 0.001, 'T_0': 20, 'T_mult': 1, 'eta_min': 5e-06}
New best model with Loss: 0.6909 at epoch 1

Epoch 1/100 - Training Loss: 0.7056 - Validation Loss: 0.6909 - Accuracy: 0.5247 - Precision: 0.5160 - Recall: 0.7947 - F1 Score: 0.6257 - Time: 12.18 - Lr: 4.97e-04
Validation loss did not improve significantly

Epoch 2/100 - Training Loss: 0.6942 - Validation Loss: 0.6956 - Accuracy: 0.5400 - Precision: 0.5826 - Recall: 0.2820 - F1 Score: 0.3801 - Time: 11.47 - Lr: 4.88e-04
New best model with Loss: 0.6903 at epoch 3

Epoch 3/100 - Training Loss: 0.6905 - Validation Loss: 0.6903 - Accuracy: 0.5293 - Precision: 0.5211 - Recall: 0.7247 - F1 Score: 0.6062 - Time: 11.55 - Lr: 4.73e-04
New best model with Loss: 0.6895 at epoch 4

Epoch 4/100 - Training Loss: 0.6896 - Validation Loss: 0.68

In [ ]:
dir_models = os.listdir("./models")

In [ ]:
best_models = [path[:-3] for path in dir_models if path.endswith(".pt")]

In [ ]:
best_val_loss = [float(str(best_model.split("+")[-1:]).split("=")[1][:8]) for best_model in best_models]

In [ ]:
best_val_loss.sort()
best_val_loss[:100]